In [1]:
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
from textblob import TextBlob
import numpy as np

c:\users\princ3\anaconda3\envs\tfconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Importing the word2vec models

In [2]:
word_vec = Word2Vec.load('model.bin')    # word embedding
# char_vec = Word2Vec.load('model_c.bin')  # charector embedding

### Importing the dataset

In [3]:
df_contxt = pd.read_pickle('./input/squad_contxt.pkl') # context
df_qas = pd.read_pickle('./input/squad_qas.pkl')       # question and answers

###  Building the model

In [4]:
class DMN_QA():
    """
    the word vectors have size 100
    the max no of words in a sentence is 120
    the answers are fed as starting and ending of the span
    """
    
    def __init__(self):
        self.MAX_WORD_NO = 120
        self.WORD_VEC_LEN = 100
    
    def build(self):
        # variables and placeholders
        Passage = tf.placeholder(shape = (None,120,100))
        Question = tf.placeholder(shape = (None,60,100))
        Answer = tf.placeholder(shape=(None,2))     
        
        # model architecture
        P_OUT,P_STATE = GRU_layer(num_units=100,num_layers=120)
        Q_OUT,Q_STATE = GRU_layer(num_units=100,num_layers=60)
               
        return 0
    
    def GRU_layer(self,num_units,num_layers,input_data):
#         dropout = tf.placeholder(tf.float32)

        cells = []
        for _ in range(num_layers):
            cell = tf.contrib.rnn.GRUCell(num_units)  # Or LSTMCell(num_units)
#             cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=1.0 - dropout)
            cells.append(cell)
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
        
        return output,state
        
        
        

### Preprocessing data

In [6]:
class prep_data():
    """
    
    """
    def __init__(self,contxt = df_contxt,qas = df_qas, word_e = word_vec):
        
        self.contxt = contxt
        self.qas = qas
        self.char_e = char_e
        self.word_e = word_e

    def P2V(self,passage):
        tb_p = TextBlob(passage)
        p2v = []
        for i in tb_p.sentences:
            words = TextBlob(str(i)+" E-O-S").words
            for j in words:
                p2v.append(word_vec[j])
        return np.array(p2v)
    
    def QA2V(self,title_no,context_no,question_no):
        tb_a = TextBlob
        
        

In [7]:
TextBlob(str(TextBlob(df_contxt['context'][1]).sentences[0])).words

WordList(['Beyoncé', 'Giselle', 'Knowles-Carter', 'biːˈjɒnseɪ', 'bee-YON-say', 'born', 'September', '4', '1981', 'is', 'an', 'American', 'singer', 'songwriter', 'record', 'producer', 'and', 'actress'])

In [13]:
df_contxt

,title,title_no,context_no,context
1,Beyoncé,1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,Beyoncé,1,2,Following the disbandment of Destiny's Child i...
3,Beyoncé,1,3,"A self-described ""modern-day feminist"", Beyonc..."
4,Beyoncé,1,4,"Beyoncé Giselle Knowles was born in Houston, T..."
5,Beyoncé,1,5,Beyoncé attended St. Mary's Elementary School ...
6,Beyoncé,1,6,"At age eight, Beyoncé and childhood friend Kel..."
7,Beyoncé,1,7,The group changed their name to Destiny's Chil...
8,Beyoncé,1,8,LeToya Luckett and Roberson became unhappy wit...
9,Beyoncé,1,9,"The remaining band members recorded ""Independe..."
10,Beyoncé,1,10,"In July 2002, Beyoncé continued her acting car..."


In [11]:
df_qas

,title_no,context_no,q_no,q_id,Question,Answer_text,Answer_start,is_impossible
1,1,1,1,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s,269,False
2,1,1,2,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing,207,False
3,1,1,3,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003,526,False
4,1,1,4,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas",166,False
5,1,1,5,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s,276,False
6,1,1,6,56bf6b0f3aeaaa14008c9603,In what R&B group was she the lead singer?,Destiny's Child,320,False
7,1,1,7,56bf6b0f3aeaaa14008c9604,What album made her a worldwide known artist?,Dangerously in Love,505,False
8,1,1,8,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Mathew Knowles,360,False
9,1,1,9,56d43c5f2ccc5a1400d830a9,When did Beyoncé rise to fame?,late 1990s,276,False
10,1,1,10,56d43c5f2ccc5a1400d830aa,What role did Beyoncé have in Destiny's Child?,lead singer,290,False


In [40]:
def QA2V(context_no,question_no):
    passage = df_contxt['context'][context_no]
    question = df_qas['Question'][question_no]
    if df_qas['is_impossible'][question_no]==False:
        answer = df_qas['Answer_text'][question_no]
        beginning = TextBlob(passage[:df_qas['Answer_start'][question_no]])
        end = TextBlob(passage[df_qas['Answer_start'][question_no]+len(answer):])  # verified
        
    else:
        return zero_v
    

In [41]:
QA2V(1,1)

False
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 
 Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead si

0

In [61]:
TextBlob(str(TextBlob(df_contxt['context'][1]).sentences[0])+" E-O-S").words

WordList(['Beyoncé', 'Giselle', 'Knowles-Carter', 'biːˈjɒnseɪ', 'bee-YON-say', 'born', 'September', '4', '1981', 'is', 'an', 'American', 'singer', 'songwriter', 'record', 'producer', 'and', 'actress', 'E-O-S'])

None
